In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install deepface

In [ ]:

#replace file to mute warning
from deepface import DeepFace
import gdown
import os
import shutil
url = "https://drive.google.com/u/0/uc?id=1aS5SJCK9MLMyYPXmhAkwtn3cJhQGCbGj&export=download"
gdown.download(url, "/content/sample_data/DeepFace.py")
os.remove("/usr/local/lib/python3.7/dist-packages/deepface/DeepFace.py")
shutil.move("/content/sample_data/DeepFace.py","/usr/local/lib/python3.7/dist-packages/deepface/")


Directory  /root /.deepface created
Directory  /root /.deepface/weights created


Downloading...
From: https://drive.google.com/u/0/uc?id=1aS5SJCK9MLMyYPXmhAkwtn3cJhQGCbGj&export=download
To: /content/sample_data/DeepFace.py
100%|██████████| 27.5k/27.5k [00:00<00:00, 32.7MB/s]


'/usr/local/lib/python3.7/dist-packages/deepface/DeepFace.py'

In [7]:
from deepface import DeepFace
import gdown
import os
import shutil

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [5]:
#!unzip /content/drive/Shareddrives/AIProject/Dataset/AAFD_train_unmasked_folder.zip -d /content/sample_data
!unzip /content/drive/Shareddrives/AIProject/Dataset/AAFD_train_masked.zip -d /content/sample_data

Streaming output truncated to the last 5000 lines.
  inflating: /content/sample_data/AAFD_train_masked/07959A11_cloth.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07960A11_surgical.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07961A11_cloth.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07962A11_N95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07963A11_KN95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07964A11_KN95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07965A11_KN95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07966A11_surgical.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07967A11_N95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07968A11_N95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07969A11_KN95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07970A11_KN95.jpg  
  inflating: /content/sample_data/AAFD_train_masked/07971A11_N95.jpg  
  infla

In [9]:
from os import listdir
from os.path import join, isfile

In [11]:
def alignfolder(unmasked_path,masked_path):
  files = listdir(unmasked_path)

  count = 0;
  for file in files:
    if not os.path.isfile(masked_path+"/"+file.split("_")[0]+".jpg"):
      print(file," not exist")
      os.remove(unmasked_path+"/"+file)
      count+=1

  print('total deleted :',count)    


alignfolder('/content/sample_data/AAFD_train_masked','/content/sample_data/AAFD_train_masked_folder')
#alignfolder('/content/sample_data/asia_celebA_test_unmasked','/content/sample_data/asia_celebA_test_masked')

test_unmasked = os.listdir('/content/sample_data/AAFD_train_masked')
test_masked = os.listdir('/content/sample_data/AAFD_train_unmasked_folder')
#train_unmasked = os.listdir('/content/sample_data/asia_celebA_train_unmasked')
#train_masked = os.listdir('/content/sample_data/asia_celebA_train_masked')



print("length test_unmasked :",len(test_unmasked))
print("length test_masked :",len(test_masked))
#print("length train_unmasked :",len(train_unmasked))
#print("length train_masked :",len(train_masked))

Streaming output truncated to the last 5000 lines.
03652A31_N95.jpg  not exist
07062A67_KN95.jpg  not exist
10947A38_N95.jpg  not exist
04242A34_cloth.jpg  not exist
02174A25_surgical.jpg  not exist
10131A33_surgical.jpg  not exist
08260A17_cloth.jpg  not exist
09048A27_N95.jpg  not exist
06597A56_N95.jpg  not exist
06640A57_cloth.jpg  not exist
01491A21_surgical.jpg  not exist
12723A60_N95.jpg  not exist
07366A80_surgical.jpg  not exist
03937A32_N95.jpg  not exist
00343A06_cloth.jpg  not exist
08534A22_N95.jpg  not exist
06220A49_N95.jpg  not exist
10876A38_surgical.jpg  not exist
02814A27_KN95.jpg  not exist
08510A22_surgical.jpg  not exist
05471A42_KN95.jpg  not exist
07539A04_cloth.jpg  not exist
02002A24_N95.jpg  not exist
00720A13_KN95.jpg  not exist
09651A31_KN95.jpg  not exist
09955A33_KN95.jpg  not exist
01397A21_N95.jpg  not exist
01603A22_KN95.jpg  not exist
12135A51_surgical.jpg  not exist
10098A33_surgical.jpg  not exist
12291A54_N95.jpg  not exist
10005A33_cloth.jpg  not 

In [12]:
#shutil.rmtree("/content/sample_data/result_face")
!mkdir /content/sample_data/result_face

In [14]:
from argparse import Namespace
import time
import os
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

EXPERIMENT_DATA_ARGS = {
    "unmasked_masked": {
        "model_path": "pretrained_models/unmasked_masked.pt",
        "image_path": "notebooks/images/input_img.jpg",
        "transform": transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
    }
}

EXPERIMENT_ARGS = EXPERIMENT_DATA_ARGS["unmasked_masked"]
img_transforms = EXPERIMENT_ARGS['transform']
resize_dims = (256, 256)

In [13]:
def calc_distance(image_name,df):
  distance = -1
  for i in range(len(df['VGG-Face_cosine'])):
    if(image_name == df['identity'][i].split("/")[4] ):
      distance = df['VGG-Face_cosine'][i]    

  if(distance == -1):
    distance = 1

  return distance  

In [ ]:

def calc_distance(image_name,df):
  distance = -1
  for i in range(len(df['VGG-Face_cosine'])):
    if(image_name == df['identity'][i].split("/")[4] ):
      distance = df['VGG-Face_cosine'][i]    

  if(distance == -1):
    distance = 1

  return distance 
N95 = 0
surgical = 0
cloth = 0
KN95 = 0

from torchvision.utils import save_image
from PIL import *
resize_dims = (256, 256)
masked_dir = "/content/sample_data/AAFD_train_masked/"
db_path = "/content/sample_data/AAFD_train_unmasked_folder"
count = 0
total_accuracy = 0
for masked_image_dir in os.listdir(masked_dir):
  if not masked_image_dir.endswith(".ipynb_checkpoints"):
    count += 1
    result_dir = masked_dir + masked_image_dir
    with torch.no_grad():
      tic = time.time()
      result_dir = masked_dir + masked_image_dir
      toc = time.time()
      df = DeepFace.find(img_path=result_dir,db_path=db_path,enforce_detection=False)
      accuracy = 1 - calc_distance(masked_image_dir.split(".")[0].split("_")[0],df)
      total_accuracy += accuracy
      if(accuracy == 0):
        mask_type = masked_image_dir.split("_")[1]
        if mask_type == "N95.jpg":
          N95 +=1
        elif mask_type == "KN95.jpg":
          KN95 +=1
        elif mask_type == "surgical.jpg":
          surgical +=1
        elif mask_type == "cloth.jpg":
          cloth +=1      

      print("masked_image :",masked_image_dir,"recognize_image:",df['identity'][0].split("/")[4],"Accuracy: ",accuracy)
      print("N95 :{} KN95 :{}  surgical :{} cloth:{} over image :{}".format(N95,KN95,surgical,cloth,count))

print("Accuracy of Model over {} images is : {}".format(count,total_accuracy/count))

Streaming output truncated to the last 5000 lines.
There are  5000  representations found in  representations_vgg_face.pkl
find function lasts  5.75208592414856  seconds
masked_image : 01055A17_KN95.jpg recognize_image: 01055A17 Accuracy:  0.861942729765852
N95 :15 KN95 :20  surgical :18 cloth:19 over image :811
There are  5000  representations found in  representations_vgg_face.pkl
find function lasts  5.863187074661255  seconds
masked_image : 06252A50_N95.jpg recognize_image: 06252A50 Accuracy:  0.8206595914554866
N95 :15 KN95 :20  surgical :18 cloth:19 over image :812
There are  5000  representations found in  representations_vgg_face.pkl
find function lasts  5.680807590484619  seconds
masked_image : 06618A56_KN95.jpg recognize_image: 08604A23 Accuracy:  0.7017132368046176
N95 :15 KN95 :20  surgical :18 cloth:19 over image :813
There are  5000  representations found in  representations_vgg_face.pkl
find function lasts  5.6798882484436035  seconds
masked_image : 05096A39_cloth.jpg re

In [ ]:
#ground_truth ffhq 0.6971544336001714